In [1]:
#import pandas and sqlite
import pandas as pd
import sqlite3 as sl

In [2]:
#connecct to database object
conn = sl.connect('NYT_covid.db')
c = conn.cursor()

In [3]:
#update counties table with most recent data
counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', index_col=0, encoding='latin-1')

c.execute('DROP TABLE IF EXISTS counties')
c.execute('CREATE TABLE counties (date datetime, county str, state str, fips str, cases int, deaths int, confirmed_cases int, confirmed_deaths int, probable_cases int, probable_deaths int)')
conn.commit()

counties.to_sql('counties',conn, if_exists='replace')

In [4]:
#confirm tables present
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

deaths
mask_use
states
us
election
regions
counties


In [5]:
df = pd.read_sql('''SELECT * FROM counties LIMIT 10''',conn)
df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
5,2020-01-25,Orange,California,6059.0,1,0
6,2020-01-25,Cook,Illinois,17031.0,1,0
7,2020-01-25,Snohomish,Washington,53061.0,1,0
8,2020-01-26,Maricopa,Arizona,4013.0,1,0
9,2020-01-26,Los Angeles,California,6037.0,1,0


In [6]:
df = pd.read_sql('''SELECT * FROM regions LIMIT 10''',conn)
df

,State,census_bureau_region,census_bureau_division,standard_federal_region,circiut_court_of_appeals,bureau_economic_analysys,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Alabama,South,East South Central,4,11,Southeast,None,None,None
1,Alaska,West,Pacific,10,9,Far West,None,None,None
2,Arizona,West,Mountain,9,9,Southwest,None,None,None
3,Arkansas,South,West South Central,6,8,Southeast,None,None,None
4,California,West,Pacific,9,9,Far West,None,None,None
5,Colorado,West,Mountain,8,10,Rocky Mountain,None,None,None
6,Connecticut,Northeast,New England,1,2,New England,None,None,None
7,Delaware,South,South Atlantic,3,3,Mideast,None,None,None
8,District of Columnia,South,South Atlantic,3,DC,Mideast,None,None,None
9,Florida,South,South Atlantic,4,11,Southeast,None,None,None


In [7]:
solution = pd.read_sql('''WITH cte as (SELECT date, r.bureau_economic_analysys as 'bea_region', c.state, county, 
                    cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                    cases as 'cumulative_cases'
                    FROM counties c
                    JOIN regions r on c.state = r.state AND r.bureau_economic_analysys = 'Great Lakes'
                    ORDER BY c.state, county)
                    
                    SELECT date, bea_region, state, county, new_cases, cumulative_cases,
                    AVG(new_cases) OVER (PARTITION BY state, county ORDER BY date ASC rows 6 PRECEDING) as '7-day_avg_new'
                    FROM cte
                    ORDER BY state, county, date
                    LIMIT 25''',conn)
solution

,date,bea_region,state,county,new_cases,cumulative_cases,7-day_avg_new
0,2020-03-20,Great Lakes,Illinois,Adams,NaN,1,NaN
1,2020-03-21,Great Lakes,Illinois,Adams,0.0,1,0.000000
2,2020-03-22,Great Lakes,Illinois,Adams,0.0,1,0.000000
3,2020-03-23,Great Lakes,Illinois,Adams,0.0,1,0.000000
4,2020-03-24,Great Lakes,Illinois,Adams,0.0,1,0.000000
5,2020-03-25,Great Lakes,Illinois,Adams,0.0,1,0.000000
6,2020-03-26,Great Lakes,Illinois,Adams,0.0,1,0.000000
7,2020-03-27,Great Lakes,Illinois,Adams,0.0,1,0.000000
8,2020-03-28,Great Lakes,Illinois,Adams,0.0,1,0.000000
9,2020-03-29,Great Lakes,Illinois,Adams,1.0,2,0.142857


In [8]:
cook = pd.read_sql('''WITH cte as (SELECT date, r.bureau_economic_analysys as 'bea_region', c.state, county, 
                    cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                    cases as 'cumulative_cases'
                    FROM counties c
                    JOIN regions r on c.state = r.state AND r.bureau_economic_analysys = 'Great Lakes'
                    ORDER BY c.state, county)
                    
                    SELECT date, bea_region, state, county, new_cases, cumulative_cases,
                    AVG(new_cases) OVER (PARTITION BY state, county ORDER BY date ASC rows 6 PRECEDING) as '7-day_avg_new'
                    FROM cte
                    WHERE state = 'Illinois' AND county = 'Cook' AND date >= '2020-09-01'
                    ORDER BY date''',conn)
cook
            

,date,bea_region,state,county,new_cases,cumulative_cases,7-day_avg_new
0,2020-09-01,Great Lakes,Illinois,Cook,415,126992,415.000000
1,2020-09-02,Great Lakes,Illinois,Cook,624,127616,519.500000
2,2020-09-03,Great Lakes,Illinois,Cook,396,128012,478.333333
3,2020-09-04,Great Lakes,Illinois,Cook,1533,129545,742.000000
4,2020-09-05,Great Lakes,Illinois,Cook,1071,130616,807.800000
5,2020-09-06,Great Lakes,Illinois,Cook,462,131078,750.166667
6,2020-09-07,Great Lakes,Illinois,Cook,544,131622,720.714286
7,2020-09-08,Great Lakes,Illinois,Cook,457,132079,726.714286
8,2020-09-09,Great Lakes,Illinois,Cook,322,132401,683.571429
9,2020-09-10,Great Lakes,Illinois,Cook,565,132966,707.714286
